In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os 
import glob as gb
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [4]:
import numpy as np 
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from collections import Counter
from keras.preprocessing.image import ImageDataGenerator
import xgboost as xgb
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [5]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith("jpg"):
            img = cv2.imread(os.path.join(folder,filename))
            img=cv2.resize(img,(256,256))              # Set whatever size you want 
            img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # to visualize  easier with matplotlib
            if img is not None:
                images.append(img)
    return images

class0=load_images_from_folder(r'../input/alpaca-dataset-small/dataset/not alpaca')
print(class0[0].shape)

class1=load_images_from_folder(r'../input/alpaca-dataset-small/dataset/alpaca')
print(class1[0].shape)




classes=[class0,class1]
x = []
for c in classes :
    for i,image in enumerate(c) :
        image=image/255  # Norm   0-1
        x.append(image)

In [6]:
labels={
    '0':'not alpaca',
    '1':'alpaca',
    
}
y = [0]*len(class0)+[1]*len(class1)

In [7]:
print('length of classes: ',len(x))
print('length of labels: ',len(y))

In [54]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0,shuffle=True)
y_test

In [9]:
X_train=np.asarray(x_train)
X_test=np.asarray(x_test)

In [10]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,15))
for index, (image, label) in enumerate(zip(X_train[0:29],y_train )):
    plt.subplot(3, 10, index+1)  # visualization  Number or Rows and Columns
    plt.imshow(image, cmap=plt.cm.Blues)
    plt.title("Training: %i\n" %label, fontsize=10)

In [11]:
import tensorflow as tf

In [12]:
# let's load VGG16 Architecture without fully connected layers, considerding only fully convolutional layers
vgg_model = VGG16(weights = 'imagenet',  include_top = False, input_shape = (256, 256, 3)) 

In [13]:
# let's make all layers non-trainable
for layer in vgg_model.layers : 
    layer.trainable = False
# now trainable parameter will be 0 in our architecture
vgg_model.summary()

In [14]:
# let's extract features from convolutional network for XBG
feature_extractor = vgg_model.predict(X_train)

In [16]:
X_train_features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [17]:
# defining XGBoost Classifier model
model = xgb.XGBClassifier()
model.fit(X_train_features, y_train)

In [18]:
feature_extractor_test = vgg_model.predict(X_test)

In [22]:
X_test_features = feature_extractor_test.reshape(feature_extractor_test.shape[0], -1)

In [23]:
prediction = model.predict(X_test_features)

In [24]:
# let's check overall accuracy
print('Accuracy : ', metrics.accuracy_score(y_test, prediction))

In [25]:
from tensorflow.keras.optimizers import Adam

In [26]:
input_tensor = tf.keras.Input(shape=(256,256,3))

base_model = tf.keras.applications.DenseNet201(
                                            input_shape = (256, 256, 3),
                                            weights="imagenet",
                                            include_top = False, 
                                            pooling = 'average'
                                            )

In [27]:
from tensorflow.keras.optimizers import Adam

In [66]:
model = tf.keras.models.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.BatchNormalization(), 
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1920, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
            ])

model.compile(
             optimizer=Adam(lr=3e-6),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [62]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("DenseNet201_replica.h5", monitor='test_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='test_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

In [63]:
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)
y_test

In [49]:
print(y_test.shape)
print(X_test.shape)

In [59]:
y_test

In [50]:
from keras.utils.np_utils import to_categorical

In [51]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [75]:
import tensorflow as tf
with tf.device("/device:GPU:0"):
    history = model.fit(X_train, y_train, batch_size=5, epochs=10,callbacks=[checkpoint,early],verbose=1)

In [72]:
print("Loss of the model is - " , model.evaluate(X_test,y_test)[0])
print("Accuracy of the model is - " ,model.evaluate(X_test,y_test)[1]*100 , "%")

In [77]:
def plot_metrics(history):
  metrics = ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    try:
      name = metric.replace("_"," ").capitalize()
      plt.plot(history.epoch, history.history[metric], label='Train')
      plt.xlabel('Epoch')
      plt.ylabel(name)
      if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
      elif metric == 'auc':
        plt.ylim([0.8,1])
      else:
        plt.ylim([0,1])
      plt.legend()
      plt.show()  
    except:
      pass
plot_metrics(model.history)

plt.title(label='Zoomed in Accuracy plot')
plt.plot(history.history["accuracy"],label='Train')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()